## Combinatorial perturbations

### This notebook contains the code for running the combinatorial perturbations identified by the minimum FVS analysis (see notebook 2.2.1_identify_FVS_subsets.ipynb)

In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import numpy as np
import maboss
import time
import datetime

In [2]:
file = "PsoriaSys"

bnd_file = file+".bnd"
cfg_file = file+".cfg"
upp_file = file+".upp"

model_init = maboss.load(bnd_file,cfg_file)

sim_init = maboss.UpdatePopulation(model_init,upp_file)
%time res_init = sim_init.run(cmaboss = True)

CPU times: user 12min, sys: 21.8 s, total: 12min 21s
Wall time: 12min 21s


In [10]:
per_pairs = pd.read_csv('output/FVS_analysis/Top20_pairs.csv',sep= " ")
per_pairs

# create vector of Receptor nodes to be removed
vec = np.array(['IL17R', 'TNFAR','IFNGR','IL10R'])

mask = per_pairs.isin(vec).any(axis=1)

# use boolean indexing to remove the rows that contain any of the values
per_pairs = per_pairs[~mask]

# print the resulting DataFrame
print(per_pairs)

    PRINCE..ModPRINCE..CheiRank Target1_tr  Target2_tr
0                            97      STAT3  Neutrophil
1                            95       IL17  Neutrophil
3                            94      STAT3        PGE2
5                            93       IL17       STAT3
6                            92       PGE2  Neutrophil
8                            89      STAT3        TNFa
9                            88        IL4  Neutrophil
10                           87       TNFa  Neutrophil
11                           86       IFNg  Neutrophil
12                           86      STAT3         IL4
13                           86       IL17        PGE2
15                           84      STAT3        IL10
16                           84      STAT3        IFNg


In [6]:
model_per = maboss.load(bnd_file,cfg_file)

for idx, row in per_pairs.iterrows():
    A = row['Target1_tr']
    B = row['Target2_tr']
    
    model_A_B = model_per.copy()
    model_A_B.mutate(A, "OFF")
    model_A_B.mutate(B, "OFF")
    
    sim_A_B = maboss.UpdatePopulation(model_A_B, upp_file, previous_run=res_init)
    res_A_B = sim_A_B.run(cmaboss=True)
    table_A_B = res_A_B.get_nodes_stepwise_probability_distribution(['KC', 'LC', 'aKC', 'Prol_KC', 'Diff_KC', 'preDiff_KC', 'pDC', 'iDC', 'M1', 'M2', 'Neutrophil', 'ILC3','Th0', 'Th1', 'Th2', 'Th17', 'Th22', 'Treg', 'Fibroblast', 'Division', 'Death'])
    table_A_B.to_csv(f'table_{A}_{B}_FVS_all_cells.csv')
    
    print(f"Done with {A}_{B}")

Done with IL17_PGE2
Done with STAT3_IL10
Done with TNFa_Neutrophil
